In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd

In [4]:
from ecephys.scoring import filter_ds

In [5]:
xr.set_options(keep_attrs=True)

In [6]:
from ecephys_analyses.data import paths, load
from ecephys import utils

In [7]:
subject = "Valentino"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
rec_condition = "recovery-sleep"
states = ["N1", "N2"]

In [8]:
bsl_spg = load.load_spectrogram(subject, experiment, bsl_condition)
bsl_hyp = load.load_hypnogram(subject, experiment, bsl_condition)
bsl_nrem_spg = filter_ds(bsl_spg, bsl_hyp, states)
bsl_prm = bsl_nrem_spg / bsl_nrem_spg.mean(dim="time") * 100

In [9]:
rec_spg = load.load_spectrogram(subject, experiment, rec_condition)
rec_hyp = load.load_hypnogram(subject, experiment, rec_condition)
rec_nrem_spg = filter_ds(rec_spg, rec_hyp, states).sel(time=slice(rec_spg.time.min(), rec_spg.time.min() + pd.to_timedelta(2, 'h')))
rec_prm = rec_nrem_spg / bsl_nrem_spg.mean(dim="time") * 100

In [10]:
nrem_spg = pd.concat([bsl_nrem_spg.to_dataframe().assign(condition="baseline"), rec_nrem_spg.to_dataframe().assign(condition="recovery")])
nrem_prm = pd.concat([bsl_prm.to_dataframe().assign(condition="baseline"), rec_prm.to_dataframe().assign(condition="recovery")])

In [11]:
def plot_spectrum(spg, y, ylabel, yscale='log'):
    fig, ax = plt.subplots(figsize=(16, 9))
    sns.lineplot(data=spg, x='frequency', y=y, hue="condition", style='condition', markers=True, dashes=False, ci=None, ax=ax)
    ax.set(ylabel=ylabel, xscale='log', yscale=yscale)

In [16]:
plot_spectrum(nrem_spg, 'mpta_wm_ref', 'CTX PSD, WM ref', yscale='linear')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
plot_spectrum(nrem_spg, 'sr_wm_ref', 'SR PSD, WM ref', yscale='linear')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plot_spectrum(nrem_prm, 'mpta_wm_ref', 'CTX PSD, WM ref, % BSL', yscale='linear')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plot_spectrum(nrem_prm, 'sr_wm_ref', 'SR PSD, WM ref, % BSL', yscale='linear')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …